In [1]:

import petpy
import pandas as pd
import urllib.request
from urllib.error import HTTPError
import time
from datetime import date
import os
from tqdm import tqdm_notebook
from ast import literal_eval
from multiprocessing.pool import ThreadPool
import re

In [70]:
# Replace xx with API key and secret
try:
    pf = petpy.Petfinder(key='Wh02Nn81tRMs4bVGMniq7f1KTNgFtPt8wCysKomBw4Bazpbqu8', 
                            secret='1033sFI9b2sgwvuQ9Q0sZKB7EjBvLQMpdLPbI86n')
    pf.breeds('dog')
except KeyError:
    pf = petpy.Petfinder(key='3iO98SzI9gtLCHpYVvl1KQJJR12pMz7WmK6f2K9QW2ErpXvDTu', 
                            secret='Vly6Ij9qJT9DYfgRI1AoemBOAHV2UVKV755sIgY0')
    

In [53]:
today_only = False

In [15]:
petpy.status

AttributeError: module 'petpy' has no attribute 'status'

In [16]:
breeds = pf.breeds('dog', return_df=True).name

In [73]:
pf


In [5]:
breeds

0                          Affenpinscher
1                           Afghan Hound
2                       Airedale Terrier
3                                 Akbash
4                                  Akita
                     ...                
270          Wirehaired Pointing Griffon
271                   Wirehaired Terrier
272    Xoloitzcuintli / Mexican Hairless
273            Yellow Labrador Retriever
274                    Yorkshire Terrier
Name: name, Length: 275, dtype: object

In [61]:
import json
full_puppy_list = []
for breed in breeds.values:
    puppy_list = [{'id': x['id'], 'breed': breed, 'photo': x['primary_photo_cropped']['large']} for x in pf.animals(animal_type='dog',age='baby', breed=breed)['animals'] if (x['primary_photo_cropped'] != []) and (x['primary_photo_cropped'] != None)]
    if len(puppy_list) > 5:
        print(breed, len(puppy_list))
        full_puppy_list += puppy_list

with open('../data/full_puppy_list.json', 'w') as outfile:
    json.dump(full_puppy_list, outfile)


#     print(breed, len(puppy_list))
#     [for i in x ]
# # (x[0]['photos'])
# breed_list
#  breed_list 
# x = [{'id': x['id'], 'breed': breed, 'photo': x['primary_photo_cropped']['large']} for x in pf.animals(animal_type='dog',age='baby', breed='Mixed Breed')['animals'] if (x['primary_photo_cropped'] != []) and (x['primary_photo_cropped'] != None)]
# x + x

Affenpinscher 19
Airedale Terrier 7
Akita 16
Alaskan Malamute 15
American Bulldog 20
American Bully 20
American Staffordshire Terrier 18
Anatolian Shepherd 19
Australian Cattle Dog / Blue Heeler 20
Australian Kelpie 20
Australian Shepherd 18
Basenji 10
Basset Hound 20
Beagle 19
Belgian Shepherd / Malinois 20
Black and Tan Coonhound 20
Black Labrador Retriever 17
Black Mouth Cur 18
Bloodhound 20
Blue Lacy 10
Bluetick Coonhound 20
Border Collie 19
Border Terrier 16
Boston Terrier 19
Boxer 19
Brittany Spaniel 8
Bull Terrier 20
Cairn Terrier 19
Cane Corso 19
Cardigan Welsh Corgi 7
Carolina Dog 16
Catahoula Leopard Dog 19
Cattle Dog 19
Chesapeake Bay Retriever 10
Chihuahua 18
Chiweenie 20
Chocolate Labrador Retriever 18
Chow Chow 20
Cocker Spaniel 20
Collie 18
Coonhound 18
Corgi 20
Dachshund 19
Dalmatian 20
Doberman Pinscher 20
Dutch Shepherd 20
English Bulldog 20
English Coonhound 8
English Pointer 13
Feist 20
Flat-Coated Retriever 8
Fox Terrier 11
Foxhound 11
French Bulldog 11
German Shep

In [79]:
with open ('../data/full_puppy_list_100perpage.json', 'r') as f:
    fpl = json.load(f) 
len(fpl)

5433

In [80]:
check_unique = set()
for puppy in fpl:
    check_unique.add(puppy['id'])
len(check_unique)

4657

In [54]:
pf.animals?

Signature:
pf.animals(
    animal_id=None,
    animal_type=None,
    breed=None,
    size=None,
    gender=None,
    age=None,
    color=None,
    coat=None,
    status=None,
    name=None,
    organization_id=None,
    location=None,
    distance=None,
    good_with_children=None,
    good_with_dogs=None,
    good_with_cats=None,
    house_trained=None,
    declawed=None,
    special_needs=None,
    before_date=None,
    after_date=None,
    sort=None,
    pages=1,
    results_per_page=20,
    return_df=False,
)
Docstring:
Returns adoptable animal data from Petfinder based on specified criteria.

Parameters
----------
animal_id : int, tuple or list of int, optional
    Integer or list or tuple of integers representing animal IDs obtained from Petfinder. When
    :code:`animal_id` is specified, the other function parameters are overridden. If :code:`animal_id`
    is not specified, a search of animals on Petfinder matching given criteria is performed.
animal_type : {'dog', 'cat', 'rabb

In [49]:
puppy_pics = [(photos['id'],photos['breeds']['primary'],photos['photos'][0]['large']) for photos in pf.animals(animal_type='dog',age='baby', pages=500)['animals'] if photos['photos'] != []]
len(puppy_pics)

9741

In [50]:
len(puppy_pics)


9741

In [51]:
pd.DataFrame(puppy_pics)[1].value_counts()

Labrador Retriever    1652
Mixed Breed            909
Shepherd               622
Pit Bull Terrier       606
Terrier                544
                      ... 
Pharaoh Hound            1
Mountain Dog             1
Goldendoodle             1
Toy Fox Terrier          1
Neapolitan Mastiff       1
Name: 1, Length: 171, dtype: int64

In [54]:
downloaded_images_id = set()
def downloader(pet_id, filename, image_url):
    pet_id = filename.split('\\')[1].split('_')[0]
    if pet_id not in downloaded_images_id:
        # full_file_name = filename + '.jpg'
        downloaded_images_id.add(pet_id)
        try:
            urllib.request.urlretrieve(image_url,full_file_name)
        except (HTTPError, OSError):
            return

In [81]:
expert_senior_imgs = [fn for fn in os.listdir(f'{expert_ds_loc}/Senior') if fn.endswith('.jpg') or fn.endswith('.png')]

NameError: name 'expert_ds_loc' is not defined